# Split All Images into Classified Folders

## Set Up

In [1]:
#import all necessary libraries
import os

from pyspark.sql.functions import lit
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import IPython.display as dp
from pyspark.ml.image import ImageSchema
from sparkdl.image import imageIO

spark = SparkSession \
    .builder \
    .master("local") \
    .appName("review_and_category_analytics") \
    .config("spark.executor.memory", '8g') \
    .config('spark.executor.cores', '4') \
    .config('spark.cores.max', '4') \
    .config("spark.driver.memory",'8g') \
    .getOrCreate()

sc = spark.sparkContext

sqlCtx = SQLContext(sc)

Using TensorFlow backend.


In [2]:
#Load classification table
class_df = sqlCtx.read.\
    format("csv").\
    option("header", "true").\
    option("inferSchema", "true").\
    load("data/galaxyClasses.csv")

class_df.show(10)

+--------+------+----+------+
|GalaxyID|Smooth|Edge|Spiral|
+--------+------+----+------+
|  100008|     0|   0|     1|
|  100023|     0|   0|     1|
|  100053|     1|   0|     0|
|  100078|     1|   0|     0|
|  100090|     1|   0|     0|
|  100122|     1|   0|     0|
|  100123|     0|   0|     0|
|  100128|     1|   0|     0|
|  100134|     0|   0|     1|
|  100143|     0|   1|     0|
+--------+------+----+------+
only showing top 10 rows



### Create List of GalaxyID for Each Type of Galaxy

In [23]:
#Find ID's of each kind of galaxy type
smoothID = class_df.where(F.col("smooth") == 1).select('galaxyID').rdd.map(lambda x: x[0]).collect()
edgeID = class_df.where(F.col("edge") == 1).select('galaxyID').rdd.map(lambda x: x[0]).collect()
spiralID = class_df.where(F.col("spiral") == 1).select('galaxyID').rdd.map(lambda x: x[0]).collect()
otherID = class_df.where((F.col("edge") == 0) & (F.col("smooth") == 0) & (F.col("spiral") == 0)).select('galaxyID').rdd.map(lambda x: x[0]).collect()

In [24]:
#Verify that there are no leftover images (should return 0)
len(smoothID) + len(edgeID) + len(spiralID) + len(otherID) - class_df.count()

0

### Move Photos to Appropriate Folder

In [39]:
#Create file directories of all smooth galaxies

smooth_dir = [] #blank list of file directories for all smooth galaxies

#loop through smooth ID's and generate corresponding file directory
for smooth in smoothID:
    smooth_dir.append("data/images_training_rev1/" + str(smooth) + ".jpg")

In [42]:
#Move all smooth galaxies to appropriate folder
for img in smooth_dir:
    !mv {smooth_dir[img]} data/galaxy_images_classified/smooth/

mv: cannot stat '{smooth_dir[img]}': No such file or directory
mv: cannot stat '{smooth_dir[img]}': No such file or directory
mv: cannot stat '{smooth_dir[img]}': No such file or directory
mv: cannot stat '{smooth_dir[img]}': No such file or directory
mv: cannot stat '{smooth_dir[img]}': No such file or directory
mv: cannot stat '{smooth_dir[img]}': No such file or directory
mv: cannot stat '{smooth_dir[img]}': No such file or directory
mv: cannot stat '{smooth_dir[img]}': No such file or directory
mv: cannot stat '{smooth_dir[img]}': No such file or directory
mv: cannot stat '{smooth_dir[img]}': No such file or directory
mv: cannot stat '{smooth_dir[img]}': No such file or directory
mv: cannot stat '{smooth_dir[img]}': No such file or directory
mv: cannot stat '{smooth_dir[img]}': No such file or directory
mv: cannot stat '{smooth_dir[img]}': No such file or directory
mv: cannot stat '{smooth_dir[img]}': No such file or directory
mv: cannot stat '{smooth_dir[img]}': No such file or di

UnboundLocalError: local variable 'child' referenced before assignment

In [ ]:
fs = os.listdir("data/images_training_rev1/") #get list of image file names
fs.sort() #sort by file name

images = [] #create blank list which will store image objects

for ea in fs: #for each image in the list of images
    #add images object to the list of images
    images.append(dp.Image(filename="data/images_training_rev1/" + ea, format="png"))